In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import json
from ingest import parse_tags
from query import query_events_db
from nostr_sdk import Keys, Client, EventBuilder, Filter, PublicKey, EventId, Options
from datetime import timedelta
import time

from sqlalchemy import create_engine, Column, BigInteger, String
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.orm import declarative_base, sessionmaker



In [ ]:
engine = create_engine(
    "postgresql://postgres@localhost:5432/postgres"
)

In [ ]:
keys = Keys.generate()
# print(keys.public_key().to_bech32())
opts = Options().timeout(timedelta(seconds=1000))
client = Client.with_opts(keys, opts)
# client = Client(keys)
client.add_relay("wss://relay.damus.io")
client.add_relay("wss://nostr.oxtr.dev")
client.add_relay("wss://nostr.openchain.fr")
client.connect()
# PublicKey.from_hex('031ea306188fee631a86f1a0e49cee8f3d62592b2e7be5f8eb1788e327db340a').to_bech32()

In [ ]:
filter = Filter().limit(50000)
all_events = client.get_events_of([filter], None)
df_all = pd.DataFrame([json.loads(event.as_json()) for event in all_events]).set_index("id")

In [ ]:
len(df_all)

In [ ]:
pd.to_pickle(df_all, 'df_all.pkl')

In [ ]:
df_all = pd.read_pickle('df_all.pkl')
# df_all.to_sql('events', engine, if_exists="replace", dtype={'tags': JSONB,})

In [ ]:
df_reply, df_mention = parse_tags(df_all)

In [ ]:
df_reply

In [ ]:
df_reply.to_sql('replys', engine, if_exists="replace")
df_mention.to_sql('mentions', engine, if_exists="replace")

In [ ]:
df_mention['marker'].unique()

In [ ]:
EventId.from_hex('3f770d65d3a764a9c5cb503ae123e62ec7598ad035d836e2a810f3877a745b24').to_bech32()

In [ ]:
PublicKey.from_hex('3f770d65d3a764a9c5cb503ae123e62ec7598ad035d836e2a810f3877a745b24').to_bech32()

In [ ]:
df_mention.iloc[0]['event_id']

In [ ]:
# events = grp.sort_values('count', ascending=False)['event_id'].values
# for id in events:
#     if id in df_all.index:
#         print(id)
# c627bc6d5fd2d75b2d9e79be9bc3a1de340536f37db8bbc828cceec904a37502
    

In [ ]:
grp = df_reply.groupby('event_id').size().reset_index(name='count')
grp.sort_values('count', ascending=False)['event_id'].iloc[0]

In [ ]:
df_mention[(df_mention.event_id == 'f30db689b6f42738fb42ecc56dba24f7991c726cf002e497d006c72841b44d51') & (df_mention.kind == 9735)].iloc[0]['source_id']
# array([   7, 9735,    3])

In [ ]:
EventId.from_hex(df_mention[df_mention.event_id == 'f30db689b6f42738fb42ecc56dba24f7991c726cf002e497d006c72841b44d51']['source_id'].iloc[1]).to_bech32()

In [ ]:
EventId.from_hex('4f8cdbd72cc7690bd4c25cf9f6e8edeb1bea02dbccaa539a5538b4ee6e130f8a').to_bech32()

In [ ]:
df_all.loc['c627bc6d5fd2d75b2d9e79be9bc3a1de340536f37db8bbc828cceec904a37502']

In [ ]:
orig_id = "c627bc6d5fd2d75b2d9e79be9bc3a1de340536f37db8bbc828cceec904a37502"

In [ ]:
df_all

In [ ]:
# thread
df_reply[df_reply.event_id == orig_id].sort_values('created_at', ascending=True)['source_id'].iloc[0]

In [ ]:
pablo = "npub1l2vyh47mk2p0qlsku7hg0vn29faehy9hy34ygaclpn66ukqp3afqutajft"
pk = PublicKey.from_bech32(pablo)
filter = Filter().author(pk.to_hex()).limit(50000)

events_pablo = client.get_events_of([filter], timedelta(seconds=1000))
df_pablo = pd.DataFrame([json.loads(event.as_json()) for event in events_pablo]).set_index("id")

In [ ]:
def get_events(npub):
    # Subtract 7 days from the current date and time
    start_ts = (datetime.now() - timedelta(days=7)).timestamp()
    # timestamp = int(seven_days_ago.timestamp())
    pk = PublicKey.from_bech32(npub)
    filter = Filter().author(pk.to_hex()).since(start_ts)
    event = client.get_events_of([filter], timedelta(seconds=1000))

In [ ]:
df_pablo = pd.read_pickle('pablo.pkl')

In [ ]:
# pd.to_pickle(df_pablo, 'pablo.pkl')
len(df_pablo)

In [ ]:
temp = df_pablo.loc['5ff739f0d1998ed71a52e45d043a1a98d95aa4cac4072d7046cc01df448b5227'].iloc[0]['tags'][0]

In [ ]:
# df.to_sql('events', engine, if_exists="replace", dtype={
#     'tags': JSONB,
#     'tags_relay_url': JSONB,
# })

# df_e.to_sql('replys', engine, if_exists="replace")
# df_p.to_sql('mentions', engine, if_exists="replace")

In [ ]:
df_pablo

In [ ]:
grp = df_e.groupby('event_id').size().reset_index(name='count')

In [ ]:
grp.sort_values('count', ascending=False)['event_id'].iloc[2]

In [ ]:
# EventId.from_hex('ef53730b18bc0a77db1078f66bebe516a13e10e036357e2750e7890da1ae047c').to_bech32()
EventId.from_hex('b6b186d3a97cb621ada7ffea9f26589c32643802b012024f4a1f2bc850faef5b') # should be nevent1 not note1


In [ ]:
EventId.from_hex('b6b186d3a97cb621ada7ffea9f26589c32643802b012024f4a1f2bc850faef5b').to_bech32()

In [ ]:
df_e[df_e.event_id == 'a6f436a59fdb5e23c757b1e30478742996c54413df777843e0a731af56a96eea']['source_id'].iloc[0]

In [ ]:
results = pd.read_sql("select * from replys", engine)

In [ ]:
df_e.head()

In [ ]:
results

In [ ]:
df_e['e_col4'].unique()

In [ ]:
df_p['p_col3'].unique()

In [ ]:
df_pablo.head()

In [ ]:
tags_count = []
for item in df_pablo['tags']:
    print(item)
    break
# result = [(lst[0], len(lst)) if lst else (None, 0) for lst in df_pablo['tags']]

# print(result)

In [ ]:
result[0]

In [ ]:
df_pablo.iloc[1]['tags']

In [ ]:
# def query_events(client, kind=None, num_limit=1000):
filter = Filter().limit(50000)
events = client.get_events_of([filter], timedelta(seconds=1000))
df = pd.DataFrame([json.loads(event.as_json()) for event in events]).set_index("id")

In [ ]:
out = df['tags'].apply(lambda x: x.keys() if x else None)

In [ ]:
df.tags.iloc[0]

In [ ]:
from collections import defaultdict
all_tags = defaultdict(int)
for row in out.values:
    if row is None:
        continue
    for item in list(row):
        all_tags[item] += 1

In [ ]:
all_tags

In [ ]:
len(all_tags)

In [ ]:
df.iloc[0]['pubkey']

In [ ]:
# Using the `size` method which returns the count of each group
grouped = df.groupby('pubkey').size().reset_index(name='count')

# OR using the `count` method
# grouped = df.groupby('kind')['kind'].count().reset_index(name='count')


In [ ]:
grouped.sort_values('count', ascending=False).head(10)

In [ ]:
# pd.to_pickle(df, 'events_50000.pkl')
# df = pd.read_pickle('events_100.pkl')

In [ ]:
PublicKey.from_hex(df_pablo.pubkey.unique()[0]).to_bech32()

In [ ]:
for i in range(20):
    print(df_pablo.iloc[i]['tags'])

In [ ]:
df

In [ ]:
df_pablo.iloc[9]['tags']

In [ ]:
df_pablo.iloc[13]['tags']

In [ ]:
df_pablo.head()

In [ ]:
df_pablo.to_sql('pablo', engine, if_exists="replace", dtype={'tags': JSONB})

In [ ]:
# df = query_events(client)
# loses association with key (e or p may have different relay url)
df["tags_relay_url"] = df["tags"].apply(
    lambda x: {item[0]: item[2] for item in x if len(item) > 2} or None
)
df['tags'] = df['tags'].apply(lambda x: {item[0]: item[1] if len(item) > 1 else None for item in x} or None)

df.to_sql('events', engine, if_exists="replace", dtype={
    'tags': JSONB,
    'tags_relay_url': JSONB,
})

# sql_query = "SELECT * FROM events"
# sql_query = "SELECT * FROM events WHERE tags ? 't'"
# sql_query = "SELECT * FROM events;"

# df_results = pd.read_sql(sql_query, engine)


In [ ]:
# Create a SQLAlchemy engine and session factory
Session = sessionmaker(bind=engine)

# Define the ORM class for the 'events' table
Base = declarative_base()

class Events(Base):
    __tablename__ = "events"
    id = Column(String, primary_key=True)
    created_at = Column(BigInteger)
    kind = Column(BigInteger)
    tags = Column(JSONB)
    tags_relay_url = Column(JSONB)
    pubkey = Column(String)
    content = Column(String)
    sig = Column(String)


In [ ]:
# Create a session and query the table using the ORM
with Session() as session:
    results = session.query(Events).all()

# Convert the results into a pandas DataFrame
df = pd.DataFrame([r.__dict__ for r in results])

# Removing unnecessary meta-data related columns
# df = df.drop('_sa_instance_state', axis=1)

print(df)

In [ ]:
npub = "npub1dergggklka99wwrs92yz8wdjs952h2ux2ha2ed598ngwu9w7a6fsh9xzpc"
pk = PublicKey.from_bech32(npub)
pk.to_hex()

In [ ]:
event_id = EventId.from_hex(df.loc[0, 'id'])
event_id.to_bech32()

In [ ]:
event_id.to_bech32()

In [ ]:
type(keys), type(keys.public_key()), type(keys.public_key().to_bech32())

In [ ]:
# sql_query = "SELECT * FROM events"
# sql_query = "SELECT * FROM events WHERE tags ? 't'"
sql_query = "SELECT * FROM events WHERE tags->>'t' = 'sm41623576'";

df_results = pd.read_sql(sql_query, engine)

In [ ]:
# create kind name -> kind val dict
# handle 3rd field (recommended relay url optional) in tag list (see nips1)
# get events for all kinds
# get zaps
# large query

In [ ]:
def query_zaps():
    filter = Filter().kind(9735).limit(10)
    events = client.get_events_of([filter], timedelta(seconds=10))
    return pd.DataFrame([json.loads(event.as_json()) for event in events])